In [118]:
import pandas as pd
import sklearn as sk
import cPickle as p
from sklearn.neighbors import KNeighborsRegressor
from sklearn.cross_validation import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
import dill

#define load the data loading method
def load_json_yelp(filepath):
    with open(filepath, 'rb') as f:
        data = f.readlines()
        data = map(lambda x: x.rstrip(), data)
        data_json_str = "[" + ','.join(data) + "]"
        data_df = pd.read_json(data_json_str)
    return data_df

#the place for class definition
class  Estimator(sk.base.BaseEstimator, sk.base.RegressorMixin):
    
    
    def fit(self,X,y):
        #X_trans = self.transformer.fit(X,y).transform(X)
        self.est = KNeighborsRegressor()
        test_param = {'n_neighbors':[5,7,9,11,13,15]}
        gscv = GridSearchCV(self.est, test_param)
        gscv.fit(X,y)
        self.est = gscv.best_estimator_
        #self.est.fit(X,y)
        #pass
        return self
    
    def score(self,X,y):
        #X_trans = self.transformer.transform(X)
        return self.est.score(X_test,y)
    
    def predict(self, X):
        #X_trans = self.transformer.transform(X)
        return self.est.predict(X)

#here is the place for trnasformer
class Transformer(sk.base.BaseEstimator, sk.base.TransformerMixin):
    def __init__(self):
        pass
        
    def fit(self,X,y=None):
        return self
    
    def transform(self,X):
        #if type(X) is list:
        #   return [[i["latitude"], i["longitude"]] for i in X]
        #el
        if type(X) == dict:
            return [[X["latitude"], X["longitude"]]]
        else:
            return X[['latitude','longitude']]
            #return X


#data loading
filepath = 'yelp_train.json'
yelp_data = load_json_yelp(filepath)

#use the transformer
X = yelp_data
y = yelp_data['stars']
#the place for generating cross validation group
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
#tuned_variable = {'estimate__n_neighbors':[5,7,9,11,13,15]}
pipeline = Pipeline([('getcols',Transformer()),('estimate',Estimator())])


In [120]:
presult = pipeline.fit(X,y)
#f = file('latlon_model','wb')
#dill.dump(presult,f)
#f.close()
#f = file('latlon_model','rb')
#a = dill.load(f)
#f.close()
#print a

with open('latlon_model','wb') as output:
    dill.dump(presult,output)

with open("latlon_model", 'rb') as in_strm:
    datastruct = dill.load(in_strm)

print datastruct

rr = datastruct.predict(X)
print rr
print len(rr)
print 'done'

Pipeline(steps=[('getcols', Transformer()), ('estimate', Estimator())])
[ 4.          3.93333333  3.83333333 ...,  3.96666667  4.06666667
  3.66666667]
37938
done


In [122]:
grid_search = GridSearchCV(pipeline, tuned_variable)

print 'grid searching..'
#grid_search.fit(X_train, y_train)

#print("Best score: %0.3f" % grid_search.best_score_)
#print("Best parameters set:")
#best_parameters = grid_search.best_estimator_.get_params()
#for param_name in sorted(tuned_variable.keys()):
    #print("\t%s: %r" % (param_name, best_parameters[param_name]))
#y_predict = grid_search.predict(X)

#pipeline_best = grid_search.best_estimator_

#f = file('latlon_model','wb')
#p.dump(pipeline_best,f)
#f.close()

grid searching..
